In [15]:
#-- rescaling 50x50 surface into 500x500 i.e 1x1px in 50x50 is 10x10px in 500x500
#-- to do snake body and food should not have same coordinate, game over after collision and pause restart button
import pygame, sys, random
import math as m
from pygame.locals import*
pygame.init()
FPS = 5
fpsclock = pygame.time.Clock()
WINWIDTH = 300
WINHEIGHT = 300
SURFACEWIDTH = 15
SURFACEHEIGHT = 15

In [16]:
class Snake:
        def __init__(self,DISPLAYSURF):
            self.size = 3
            self.rectsize = 1  # draw snake on pixel
            self.node = [[0,0],[3,3],[2,3],[1,3]]
            self.tail = self.node[self.size]
            self.color = (0,255,0)
            self.direction = 'down'
            self.surface = DISPLAYSURF
            self.SPEED =  1 # one pixel at a time
            self.RESCALEFACTOR = 20 # 1px in 50x50 surface is 10px in 500x500 window
            temp = self.size
            while temp != 0:
                self.drawrect(self.node[temp][0],self.node[temp][1],self.rectsize,self.rectsize) 
                temp -= 1
                
        def updateS(self):
                previous = [self.node[1][0],self.node[1][1]]
                if self.direction == 'right' :
                      previous[0] += self.SPEED
                elif self.direction == 'left':
                      previous[0] -= self.SPEED
                elif self.direction == 'up':
                      previous[1] -= self.SPEED
                elif self.direction == 'down':
                      previous[1] += self.SPEED 
                self.node.insert(1,previous)
                self.node.pop()
        
        def addtail(self):
                newtail = [self.tail[0],self.tail[1]]
                if self.direction == 'right' :
                      newtail[0] -= 1
                elif self.direction == 'left':
                      newtail[0] += 1
                elif self.direction == 'up':
                      newtail[1] += 1
                elif self.direction == 'down':
                      newtail[1] -= 1
                self.node.insert(self.size+1,newtail)
                self.size += 1
                self.tail = self.node[self.size]
                
        def collidedItself(self):
            for xy in self.node[2:]:
                if self.node[1][0] == xy[0] and self.node[1][1] == xy[1]:
                    return True
            return False
        
        def outOfbound(self):
            if (self.node[1][0]*self.RESCALEFACTOR >=  WINWIDTH) or (self.node[1][0] < 0) or (self.node[1][1]*self.RESCALEFACTOR >= WINHEIGHT) or (self.node[1][1] < 0):
                return True
            return False
                
        def drawSnake(self):
            sizes = 1
            while sizes != self.size + 1:
                self.drawrect(self.node[sizes][0],self.node[sizes][1],self.rectsize,self.rectsize)
                sizes += 1
                
        def drawrect(self,x,y,xsize,ysize):
                pygame.draw.rect(DISPLAYSURF,(0,255,0),(x*self.RESCALEFACTOR,y*self.RESCALEFACTOR,xsize*self.RESCALEFACTOR,ysize*self.RESCALEFACTOR))

In [17]:
class Food:
    def __init__(self,WINWIDTH,WINHEIGHT,MINWIDTH,MINHEIGHT):
        self.maxwidth = WINWIDTH - 2
        self.maxheight = WINHEIGHT - 2
        self.minwidth =  MINWIDTH + 2
        self.minheight = MINHEIGHT + 2
        self.size = 1
        self.RESCALEFACTOR = 20
        self.color = (255,0,0)
        self.currentX =  -1
        self.currentY =  -1
        self.generateXY()
    
    def generateXY(self):
        self.currentX = random.randint(self.minwidth,self.maxwidth)
        self.currentY = random.randint(self.minheight,self.maxheight)
    
    def drawFood(self,surface):
        #print("food location is",self.currentX,self.currentY)
        self.drawrect(surface,self.currentX,self.currentY,self.size,self.size)
    
    def drawrect(self,surface,x,y,xsize,ysize):
        pygame.draw.rect(surface,self.color,(x*self.RESCALEFACTOR,y*self.RESCALEFACTOR,xsize*self.RESCALEFACTOR,ysize*self.RESCALEFACTOR))
        
        
    

In [18]:
WHITE = (255,255,255)

In [19]:
def rungame(fil):
    snake = Snake(DISPLAYSURF)
    food = Food(SURFACEWIDTH,SURFACEHEIGHT,0,0)
    #-- loop to start game after space is pressed
    space = False
    while True:
        for event in pygame.event.get():
            if event.type == KEYDOWN:
                if event.key == K_UP:
                    space = True
                    break
        if space:
            break
    score = 0
    string = str(score)
    string = "Snake Game Score:" + string
    pygame.display.set_caption(string)
    #-- main game loop
    while True:
        DISPLAYSURF.fill(WHITE)
        
        
        for event in pygame.event.get():
            if event.type == QUIT:
                fil.close()
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if event.key == K_UP:
                    if snake.direction != 'down':
                        snake.direction = 'up'
                if event.key == K_DOWN:
                    if snake.direction != 'up':
                        snake.direction = 'down'
                if event.key == K_LEFT:
                    if snake.direction != 'right':
                        snake.direction = 'left'
                if event.key == K_RIGHT:
                    if snake.direction != 'left':
                        snake.direction = 'right'
                        
        obstacle = obstacles(snake)
        angle = getOrthogonalAngle(snake ,food)
        distance = distanceBetweenSnakeAndFood(snake, food)
        print(obstacle[0],obstacle[1],obstacle[2],obstacle[3],angle,distance,snake.size,number(snake.direction), sep=",", file=fil)
        
        
        if (snake.node[1][0] == food.currentX) and (snake.node[1][1] == food.currentY):
            #print("food found")
            snake.addtail()
            food.generateXY()
            score += 1
            string = "Snake Game Score:" + str(score)
            pygame.display.set_caption(string)
        
        
        food.drawFood(DISPLAYSURF)
        snake.updateS()
        if (snake.collidedItself()):
            pygame.display.set_caption("Game Over collided")
            break
        if (snake.outOfbound()):
            pygame.display.set_caption("Game Over out of bound")
            break
        snake.drawSnake()
        pygame.display.update()
        fpsclock.tick(FPS)

In [20]:
def number(direction):
    if direction == 'up':
        return -1
    elif direction == 'down':
        return +1
    elif direction == 'left':
        return -3
    return +3

In [21]:
def getOrthogonalAngle(snake ,food):
    headx, heady = snake.node[1][0], snake.node[1][1]

    foodx,foody = food.currentX, food.currentY

    base = foodx - headx
    perpendicular = foody - heady

    hypotenuse = m.sqrt(base**2 + perpendicular**2)+0.00001

    angle = m.degrees(m.asin(perpendicular/hypotenuse))%90

    if snake.direction == "right":
        if base >= 0 and perpendicular <= 0:   angle = angle + 0
        elif base <= 0 and perpendicular <= 0: angle = angle + 90
        elif base <= 0 and perpendicular >= 0: angle = angle + 90
        else:                                  angle = angle + 0
    elif snake.direction == "up":
        if base >= 0 and perpendicular <= 0:   angle = angle + 0
        elif base <= 0 and perpendicular <= 0: angle = angle + 0
        elif base <= 0 and perpendicular >= 0: angle = angle + 90
        else:                                  angle = angle + 90
    elif snake.direction == "left":
        if base >= 0 and perpendicular <= 0:   angle = angle + 90
        elif base <= 0 and perpendicular <= 0: angle = angle + 0
        elif base <= 0 and perpendicular >= 0: angle = angle + 0
        else:                                  angle = angle + 90
    else:
        if base >= 0 and perpendicular <= 0:   angle = angle + 90
        elif base <= 0 and perpendicular <= 0: angle = angle + 90
        elif base <= 0 and perpendicular >= 0: angle = angle + 0
        else:                                  angle = angle + 0

    return m.radians(angle-90)/(m.pi/2)

In [22]:
def distanceBetweenSnakeAndFood(snake, food):
    headx, heady = snake.node[1][0], snake.node[1][1]

    foodx, foody = food.currentX, food.currentY

    base = abs(foodx - headx)
    perpendicular = abs(foody - heady)

    hypotenuse = m.sqrt(base**2 + perpendicular**2)
    return hypotenuse/15

In [23]:
def obstacles(snake):
    head = [snake.node[1][0], snake.node[1][1]]
    up = 0
    down = 0
    left = 0
    right = 0
    if (head[0]+2)*snake.RESCALEFACTOR >= WINWIDTH:
        right = 1
    if (head[0]-2)*snake.RESCALEFACTOR < 0:
        left = 1
    if (head[1]+2)*snake.RESCALEFACTOR >= WINHEIGHT:
        down = 1
    if (head[1]-2)*snake.RESCALEFACTOR < 0:
        up = 1
    
    for xy in snake.node[2:]:
        if (head[0]) == xy[0] and (head[1]-1) == xy[1]:
            up = 1
        elif (head[0]) == xy[0] and(head[1]+1) == xy[1]:
            down = 1
        elif (head[0]-1) == xy[0] and (head[1]) == xy[1]:
            left = 1
        elif (head[0]+1) == xy[0] and (head[1]) == xy[1]:
            right = 1
            
    return (up,down,left,right)
    

In [24]:

DISPLAYSURF = pygame.display.set_mode((WINWIDTH,WINHEIGHT),0,32)
fil = open("newdata3.csv", "a+")
#fil.write("up,down,left,right,angle,distance,size,decision\n")
while True:
    rungame(fil)

SystemExit: 

/home/mahadev/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
